Scratch code for generalization to n_variant a/b test + analysis

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.ticker as mtick

from matplotlib import style
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests as mult_test
from tqdm.auto import tqdm

In [2]:
# additional imports 
sys.path.append('../../ab_testing_utils/')

In [122]:
import conversion_rate_utils
import sim_utils

In [121]:
from importlib import reload

In [4]:
# Set pandas preferences
pd.options.display.max_columns=500
pd.options.display.max_colwidth=500
pd.options.display.max_rows=500

In [5]:
# Set plot style
style.use('fivethirtyeight')

In [6]:
# Initialize tqdm for pandas
tqdm.pandas()

We'll continue to work in units of weeks for the experiment run times.  Suppose that we already know the average baseline conversion rate, as well as the average number of daily visitors from historical data. There could be variance in these estimates, but we'll ignore that problem here for now

In [8]:
daily_num_observations = 10000
monthly_num_observations = daily_num_observations * 7 * 4
baseline_conversion_rate = 0.05

In [7]:
number_of_weeks_for_experiment = 4
number_of_days_for_experiment = number_of_weeks_for_experiment * 7

In [120]:
# Simulate a variant test

In [123]:
sim_ = sim_utils.SimulateABTest()

In [125]:
df_ab = sim_.run_sim(daily_num_observations=daily_num_observations, 
             number_of_days_for_experiment=number_of_days_for_experiment, 
             expected_conversion_rates=[baseline_conversion_rate, 0.1, 0.25], 
             group_col='group', 
             p_vals='equal')

Simulated group sizes: 

treatment_2    0.334503
control        0.332976
treatment_1    0.332522
Name: group, dtype: float64


Simulated daily group sizes: 

day  group      
0    control        0.340257
     treatment_2    0.334125
     treatment_1    0.325618
1    treatment_1    0.336545
     treatment_2    0.334459
     control        0.328996
10   treatment_2    0.336997
     treatment_1    0.332401
     control        0.330602
11   treatment_1    0.337688
     control        0.332602
     treatment_2    0.329710
12   treatment_1    0.336662
     treatment_2    0.335364
     control        0.327974
13   treatment_2    0.340688
     control        0.334514
     treatment_1    0.324798
14   treatment_2    0.339081
     treatment_1    0.338284
     control        0.322635
15   control        0.340038
     treatment_2    0.332968
     treatment_1    0.326994
16   control        0.338755
     treatment_2    0.335522
     treatment_1    0.325722
17   treatment_1    0.338332
     control 

In [126]:
df_ab.head()

,day,units,group,conversion_probability,conversion
0,0,1,control,0.05,0
1,0,1,treatment_2,0.25,0
2,0,1,control,0.05,0
3,0,1,control,0.05,0
4,0,1,treatment_1,0.10,1
